# **ĐỒ ÁN NHẬP MÔN THỊ GIÁC MÁY TÍNH**







**THÀNH VIÊN NHÓM:**

> 1.   *Phạm Bùi Nhật Huy*
> 2.   *Nguyễn Thị Kim Anh*
> 3.   *Võ Minh Trí*

Đề tài: Hệ thống gợi ý phối trang phục (Dress up Recommendation System)







1.   **Mô tả đồ án:**
    
        Đồ án gồm có 2 phần chính là: Ứng dụng các mô hình học để nhận diện các đối tượng quan tâm (Computer Vision) để phục vụ cho việc xử lý ảnh và Xử lý ảnh (Image Processing) rút trích thông tin, đặc trưng bức ảnh sử dụng cho các mục đích tính toán.
2.   **Mục đích đồ án:**

        Sử dụng các kiến thức về xử lý ảnh và các mô hình phân loại đã học trên lớp học ứng dụng vào việc gợi ý trang phục cho người mặc một cách nhanh chóng.
3.   **Thao tác liên quan đến Computer Vision:**

  *   Phân loại đối tượng:
    
      Ở bước này, phân lớp các bức ảnh. Ta có 6 lớp chính:

      *   Top: Áo.
      *   Bottom: Quần, váy.
      *   Dress.
      *   Hat
      *   Bag&Backpack.
      *   Shoes
      
      Ngoài ra, ta phân các lớp trên vào 3 lớp lớn khác là:

      *   Male: Trang phục cho nam (Top, Bottom, Hat, Bag&Backpack, Shoes)
      *   Female: Trang phục cho nữ (Top, Bottom, Dress, Hat, Bag&Backpack, Shoes)
      *   Unisex: Trang phục mà cả nam và nữ đều có thể mặc

  *   Sử dụng MASK-RCNN cho việc tách đối tượng ra khỏi nền:

        * Phân loại bức ảnh đầu vào thuộc lớp nào.

        * Ứng với từng lớp, ta sẽ có một model riêng phù hợp để tìm ra contour của đối tượng thuộc lớp đã xác định.

        * Dựa vào contour, ta xác định bounding box để crop đối tượng đó: Điểm cực trái, cực phải, cực trên, cực dưới trong tập hợp điểm mà contour.

        * Contour mà mô hình trả lại là Mask cho một đối tượng được tìm thấy.

        * Tách đối tượng dựa trên Mask đã tìm. Sau đó, ta đặt đối tượng đó trên nền ảnh trắng (kích thước nền ảnh này dựa trên kích thước ảnh cũ).
        
        * Crop ảnh sát đối tượng bằng bounding box đã xác định ở trên.
4.   **Thao tác xử lý ảnh cho phần gợi ý phối trang phục:**

  * Tạo tập dữ liệu cơ sở cho việc gợi ý:
    
      * Ở bước này lập dữ liệu dựa trên những bộ đồ đã được phối (Tập hợp các ảnh lẻ bao gồm áo, quần, giày,... đi chung với nhau).
      
      * Mỗi bộ đồ như thế, ta rút trích đặc trưng về cạnh, màu sắc. Lưu ý rằng các món đồ ứng với bộ đồ đều đã được tách ra khỏi nền ảnh phức tạp dựa trên mô hình MASK-RCNN phía trên.

      * Tính toán Histogram dựa trên 2 đặc trưng này rồi lưu lại để phục vụ cho việc tính toán độ tương đồng.
      
      * Tập dữ liệu cơ sở này bao gồm nhiều bộ đồ.

  * Thao tác gợi ý trang phục dựa trên những món đồ được đưa vào:

      * Đầu vào là các bức ảnh chứa món đồ, ta phân loại chúng và tách chúng ra khỏi nền dựa trên mô hình MASK-RCNN.
      
      * Gán nhán cho chúng dựa trên 6 nhãn đơn giản (Top, Bottom, Dress, Hat, Bag&Backpack, Shoes). Rút trính đặc trưng cạnh, màu sắc và tính toán Histogram.

      * Dựa trên Histogram, tính toán độ tương đồng của chúng so vs từng món trong từng bộ đồ thuộc tập dữ liệu cơ sở.

      * Độ tương đồng càng cao thì ta sẽ chọn món đồ đó và add vào tập dữ liệu bộ đồ và bắt đầu tìm món đồ tiếp theo phù hợp với món đồ đó.

      * Ta sẽ đề xuất 5-10 bộ đồ.

# **1. IMPORT THƯ VIỆN VÀ KẾT NỐI GGDRIVE**

In [1]:
from google.colab.patches import cv2_imshow
import cv2 as cv
import cv2
from PIL import Image, ImageDraw, ImageFilter
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation
from scipy import ndimage
from scipy.spatial import distance

import os

import numpy as np
import pandas as pd
import math

import pickle
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# **2. CÀI ĐẶT THƯ VIỆN**

In [3]:
%tensorflow_version 1.x
!pip install --upgrade h5py==2.10.0
!wget https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
!unzip Mask_RCNN_basic_1.zip
import sys
sys.path.append("/content/Mask_RCNN/mrcnn")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

TensorFlow 1.x selected.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 27.0 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
--2022-07-04 14:15:30--  https://pysource.com/extra_files/Mask_RCNN_basic_1.zip
Resolving pysource.com (pysource.com)... 104.21.67.193, 172.67.180.33, 2606:4700:3036::ac43:b421, ...
Connecting to pysource.com (pysource.com)|104.21.67.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1282557 (1.2M) [application/zip]
Saving to: ‘Mask_RCNN_basic_1.zip’

Mask_RCNN_basic_1.z 100%[===================>]   1.22M  --.-KB/s    in 0.006s  

2022-07-04 14:15:30 (202 MB/s) - ‘Mask_RCNN_basic_1.zip’ saved [1282557/1282557]

Archive:  Mask_RCNN_basic_1.zip
   creating: Mask_RCNN/
  inflating: Mask_RCNN/.gitignore    
   creating: Mask_RCNN/assets/
   cr

Using TensorFlow backend.


... done downloading pretrained model!


# **3. CÁC HÀM CƠ BẢN**

## **SHOW IMAGE**

showimage(img_src, title)

    In 1 ảnh ra ngoài kèm theo tiêu đề

show2image(img_src, img_rst, title1, title2)

    In 2 ảnh ra ngoài kèm theo tiêu đề

In [4]:
def showimage(img_src, title, figsize_=(40, 40)):
  plt.figure(figsize=figsize_)

  # show img src
  plt.subplot(1, 2, 1)
  plt.title(title)
  img_src = cv2.cvtColor(img_src.astype(np.uint8), cv2.COLOR_BGR2RGB)
  plt.imshow(img_src, interpolation='bicubic')

def show2image(img_src, img_rst, title1, title2, figsize_=(20, 20)):
  plt.figure(figsize=figsize_)
  # show img src
  plt.subplot(1, 2, 1)
  plt.title(title1)
  img_src = cv2.cvtColor(img_src.astype(np.uint8), cv2.COLOR_BGR2RGB)
  plt.imshow(img_src, interpolation='bicubic')

  # show img result
  plt.subplot(1, 2, 2)
  plt.title(title2)
  img_rst = cv2.cvtColor(img_rst.astype(np.uint8), cv2.COLOR_BGR2RGB)
  plt.imshow(img_rst, interpolation='bicubic')

def showListImg(pathList, titleList, figsize_=(20, 20)):
  plt.figure(figsize=figsize_)
  
  for i in range(0, len(pathList)):
    plt.subplot(1, len(pathList), i + 1)
    plt.title(titleList[i])
    img = cv.imread(pathList[i], cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
    plt.imshow(img, interpolation='bicubic')

## **READ IMAGE**

readImageWithPathAndNameimage (dir_image, name_image, mode = cv2.IMREAD_COLOR)

    Đọc ảnh theo đường dẫn đến nơi lưu ảnh: Đường dẫn + tên file ảnh
    Ngoài ra, có kèm theo chế độ đọc mode, có 3 chế độ:
        mode = cv2.IMREAD_COLOR (Default) - Đọc ảnh màu 3 kênh RGB
        mode = cv2.IMREAD_GRAYSCALE - Đọc ảnh trắng đen
        mode = cv2.IMREAD_UNCHANGED - Đọc ảnh mà không thay đổi gì (3 hoặc 4 kênh nếu ảnh có hỗ trợ ARGB (alpha or transparency channel)) 

In [5]:
def readImageWithPathAndNameimage (dir_image, name_image, mode = cv2.IMREAD_COLOR):
  path_filename = os.path.join(dir_image, name_image)
  img = cv.imread(path_filename, mode)
  return img

## **CACULATE HISTOGRAM**

### **TÍNH HISTOGRAM ẢNH TRẮNG ĐEN**

calHistImg(pathfilename)

In [6]:
def calHistImg(pathfilename):
  img = cv.imread(pathfilename, cv2.IMREAD_GRAYSCALE)
  hist = cv.calcHist([img],[0],None,[256],[0,256])
  size = img.shape[0]*img.shape[1]
  hist = hist / size
  return hist  

### **TÍNH HISTOGRAM ẢNH MÀU**

calcHistColorImg(pathfilename)

In [7]:
def calcHistColorImg(pathfilename):
  img = cv.imread(pathfilename, cv2.IMREAD_COLOR)
  hist = cv.calcHist([img], [0, 1, 2], None,
                     [8, 8, 8], [0, 256, 0, 256, 0, 256])
  size = img.shape[0] * img.shape[1] * img.shape[2]
  hist = hist / size
  return hist

In [8]:
def calcHistColorImg_(img, size = (800, 1300)):
  img_ = cv.resize(img, size)
  hist = cv.calcHist([img_], [0, 1, 2], None,
                     [8, 8, 8], [0, 256, 0, 256, 0, 256])
  size = img_.shape[0] * img_.shape[1] * img_.shape[2]
  hist = hist / size
  return hist

## **TÍNH TOÁN COSIN FEATURE (COLOR) TỪ ẢNH**

In [9]:
def cosinHistColor(Hist1, Hist2):
  Hist1 = Hist1.reshape(Hist1.flatten().shape[0])
  Hist2 = Hist2.reshape(Hist2.flatten().shape[0])
  return 1 - distance.cosine(Hist1, Hist2)

## **SOBEL FILTER**

sobel_filters(img)

Kernel theo trục ngang x:

    [-1  0  1
     -2  0  2
     -1  0  1]

 Kernel theo trục dọc y:

    [1  2  1 
     0  0  0  
    -1 -2 -1]

In [10]:
def sobel_filters(img):
  Sx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
  Sy = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)

  Ix = cv.filter2D(img, -1, Sx)
  Iy = cv.filter2D(img, -1, Sy)
  G = np.hypot(Ix, Iy)
  G = G / G.max() * 255
  theta = np.arctan2(Iy, Ix)

  return Ix, Iy, G, theta

## **CONVERT TO RGB IMAGE**

In [11]:
def convertRGBImg(path_img):
  im = Image.open(path_img)
  rgb_im = im.convert('RGB')
  rgb_im.save(path_img)

## **TÍNH TOÁN FEATURE (EDGE) TỪ ẢNH**

Xử lý trên ảnh grayscale để cải thiện kết quả

In [12]:
def calFeatureVector(img, size = (800, 1300)):
  img_ = cv.resize(img, size)
  _, _, G, _ = sobel_filters(img_)
  row_sum = np.sum(G, axis=1, dtype=np.float64)
  col_sum = np.sum(G, axis=0, dtype=np.float64)
  return (np.hstack((row_sum.T, col_sum)))

## **TÍNH TOÁN COSIN FEATURE (EDGE)**

In [13]:
def cosinEdge(feature1, feature2):
  return 1 - distance.cosine(feature1, feature2)

## **KIỂM TRA NOT A NUMBER**

In [14]:
def isNan(Number):
  if math.isnan(Number) == True:
    return True
  else:
    return False

# **I. PHÂN LOẠI**

## **RÚT TRÍCH FEATURE (HISTOGRAM) TỪ ẢNH**

### **ẢNH LẺ**

In [15]:
def extractFeatureImg(img_path):

  features = []
  feature = calHistImg(img_path)
  feature = feature.reshape(feature.flatten().shape[0])
  features.append(feature)

  return np.array(features)

### **MỘT TẬP ẢNH**

In [16]:
def extractFeatureImgs(path_to_files):
  
  features = []
  for file_name in os.listdir(path_to_files):
    feature = calHistImg(os.path.join(path_to_files, file_name))
    feature = feature.reshape(feature.flatten().shape[0])
    features.append(feature)

  return np.array(features)

## **PHÂN LOẠI ĐỐI TƯỢNG**

### **PHÂN LOẠI ẢNH LẺ**

classificationCloth(img_path)

In [17]:
def classificationCloth(img_path):

  model = pickle.load(open('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/knnpickle_file', 'rb'))
  feature = extractFeatureImg(img_path)
  pre = model.predict(feature)

  return pre

### **PHÂN LOẠI MỘT TẬP ẢNH**

classificationClothes(imgs_path)

In [18]:
def classificationClothes(imgs_path):

  model = pickle.load(open('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/knnpickle_file', 'rb'))
  feature = extractFeatureImgs(imgs_path)
  pre = model.predict(feature)

  return pre

# **II. TÁCH ĐỐI TƯỢNG TRANG PHỤC SANG MỘT NỀN TRẮNG**

## **DRAW MASK**

draw_mask(img, img_2, pts, color, alpha=1)

In [19]:
def draw_mask(img1, img2, pts, color, alpha=1):
  
  # img1 ảnh gốc
  # img2 là cái background trắng
  # pts tập hợp các điểm của countour dùng để vẽ Mask
  # color màu sắc của Mask
  # alpha là độ sáng của cảnh img.
  # alpha càng nhỏ thì bức ảnh img2 sẽ hiện rõ ràng hơn và che khuất img

  h, w, _ = img1.shape
  overlay = img2.copy()
  output = img1.copy()
  dim = (w, h)
  overlay = cv.resize(overlay, dim, interpolation = cv.INTER_AREA)

  cv2.fillPoly(overlay, pts, color)
  output = cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0)
  return output

## **CHỌN MÔ HÌNH MASK-RCNN PHÙ HỢP VỚI ĐỐI TƯỢNG SAU KHI PHÂN LOẠI TỪ 1 ẢNH**

chooseMaskRcnnModel_path(path_img)

In [20]:
def chooseMaskRcnnModel(path_img):
  pre = classificationCloth(path_img)
  if 'male' in pre[0]:
    sex = 'male'
  else:
    sex = 'female'

  if 'hat' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_hat.h5', sex, 'hat'
  if 'top' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5', sex, 'top'
  if 'bottom' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_bottom.h5', sex, 'bottom'
  if 'dress' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_dress.h5', sex, 'dress'
  if 'shoes' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_shoes.h5', sex, 'shoes'
  if 'bag&backpack' in pre[0]:
    return '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_bag&backpack.h5',sex, 'bag&backpack'

## **TÌM ĐỐI TƯỢNG TRONG HÌNH**

In [21]:
def detectObject(img, model):
  test_model, inference_config = load_inference_model(1,model)
  image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # Detect results
  r = test_model.detect([image])[0]
  #colors = random_colors(80)
  return r

## **VẼ MASK BAO QUANH ĐỐI TƯỢNG VÀ COMPOSITE NỀN TRẮNG VỚI ĐỐI TƯỢNG**

In [22]:
def drawContours(img1, img2, Dectect_result):
  r = Dectect_result
  mask = r["masks"][:, :, 0]
  contours = get_mask_contours(mask)
  img_1 = img1.copy()
  img_2 = img2.copy()

  cv2.polylines(img_1, [contours], True, (255,0,0), 2)
  Mask_img = draw_mask(img_1,img_2, [contours], (255,0,0))

  return Mask_img, img_1, contours

In [23]:
def drawMaskObject(Detect_Result,
                   path_img1,
                   path_img2='/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/background.jpg'):
  
  object_count = len(Detect_Result["class_ids"])
  for i in range(object_count):
    mask = Detect_Result["masks"][:, :, i]
    Contours = get_mask_contours(mask)

  img = cv.imread(path_img1)
  #for cnt in Contours:
  cv2.polylines(img, [Contours[0]], True, (255,0,0), 2)
  img2 = cv.imread(path_img2)
  img1 = draw_mask(img,img2, [Contours[0]], color=(255,0,0))
  #cv2_imshow(img)
  !rm mask.jpg
  cv.imwrite('mask.jpg', img1)

  im1 = Image.open(path_img1)
  im2 = Image.open('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/background.jpg').resize(im1.size)
  mask = Image.open('/content/mask.jpg').convert('L').resize(im1.size)
  im = Image.composite(im2, im1, mask)

  Name_file = path_img1.split(os.path.sep)[-1].split("/")
  Name_file = Name_file[len(Name_file) - 1]
  im.save(Name_file)
  img_result = cv.imread(os.path.join('/content', Name_file))
  
  return img_result, Contours

## **CROP ẢNH**

In [24]:
def Crop(img, Contours):
  # Find x,y,w,h of bounding box
  array=np.argmax(Contours[0], axis=0)
  indice_X_max,indice_Y_max=array[0],array[1]
  array=np.argmin(Contours[0], axis=0)
  indice_X_min,indice_Y_min=array[0],array[1]
  #print(contours[0][indice_X_max][0],contours[0][indice_Y_max][1])
  #print(contours[0][indice_X_min][0],contours[0][indice_Y_min][1])
  X_max,Y_max,X_min,Y_min=(Contours[0][indice_X_max][0],
                           Contours[0][indice_Y_max][1],
                           Contours[0][indice_X_min][0],
                           Contours[0][indice_Y_min][1])
  w=X_max-X_min
  h=Y_max-Y_min
  x,y=X_min,Y_min
  #print(w,h,X_min,Y_min)

  img_crop = img[y:y+h, x:x+w, :]
  return img_crop

## **MASK-RCNN CHO MỘT ẢNH**

In [25]:
def app_maskrcnn(img_path):
  
  im = Image.open(img_path)
  rgb_im = im.convert('RGB')
  rgb_im.save(img_path)

  background = '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/background.jpg'
  img = cv.imread(img_path)
  img_2 = cv.imread(background)
  width = int(img.shape[1] * 100 / 100)
  height = int(img.shape[0] * 100 / 100)
  dim = (width, height)
  img_2 = cv.resize(img_2, dim, interpolation = cv.INTER_AREA)

  model, sex, typeOutfit = chooseMaskRcnnModel(img_path)
  Detect_Result = detectObject(img, model)
  img_Result, Contours = drawMaskObject(Detect_Result, img_path, background)
  img_cropped = Crop(img_Result, Contours)

  return img_cropped, sex, typeOutfit

# **III. TẠO LẬP CƠ SỞ NỀN CHO HỆ THỐNG GỢI Ý**

## **ĐỌC FILE EXCEL**

In [26]:
df = pd.read_csv('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/Fashion Product Image Dataset/images đã phân theo từng loại/Clothes.csv')

## **HÀM RÚT TRÍCH THÔNG TIN TỪ ẢNH TRONG FILE EXCEL**

Tính Histogram trên ảnh màu, detect cạnh trên ảnh grayscale

In [26]:
def extractInfoImgs(df, id = 0):
  choosenSex = ['male', 'female', 'unisex']
  i = id
  print("Clothes " + str(i))
  tmp = []

  pathTop = os.path.join(df.at[i, 'path'], choosenSex[int(df.at[i, 'sex'])], 'top')
  pathBottom = os.path.join(df.at[i, 'path'], choosenSex[int(df.at[i, 'sex'])], 'bottom')
  pathShoes = os.path.join(df.at[i, 'path'], choosenSex[int(df.at[i, 'sex'])], 'shoes')
  pathBagBackpack = os.path.join(df.at[i, 'path'], choosenSex[int(df.at[i, 'sex'])], 'bag&backpack')
  pathHat = os.path.join(df.at[i, 'path'], choosenSex[int(df.at[i, 'sex'])], 'hat')
  tmp.append(choosenSex[int(df.at[i, 'sex'])])

  if not(isNan(df.at[i, 'top'])):
    imgName = str(int(df.at[i, 'top'])) + '.jpg'
    pathImg = os.path.join(pathTop, imgName)
    colorImg, _, _ = app_maskrcnn(pathImg)
    grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)

    hist = calcHistColorImg_(colorImg)
    edgeFeature = calFeatureVector(grayscaleImg)
    tmp.append([hist, edgeFeature])
  else:
    tmp.append(['NULL', 'NULL'])
###################################################################
  if not(isNan(df.at[i, 'bottom'])):
    imgName = str(int(df.at[i, 'bottom'])) + '.jpg'
    pathImg = os.path.join(pathBottom, imgName)
    colorImg, _, _ = app_maskrcnn(pathImg)
    grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)

    hist = calcHistColorImg_(colorImg)
    edgeFeature = calFeatureVector(grayscaleImg)
    tmp.append([hist, edgeFeature])
  else:
    tmp.append(['NULL', 'NULL'])
###################################################################
  if not(isNan(df.at[i, 'shoes'])):
    imgName = str(int(df.at[i, 'shoes'])) + '.jpg'
    pathImg = os.path.join(pathShoes, imgName)
    colorImg, _, _ = app_maskrcnn(pathImg)
    grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)

    hist = calcHistColorImg_(colorImg, (800, 400))
    edgeFeature = calFeatureVector(grayscaleImg, (800, 400))
    tmp.append([hist, edgeFeature])
  else:
    tmp.append(['NULL', 'NULL'])
###################################################################
  if not(isNan(df.at[i, 'bag&backpack'])):
    imgName = str(int(df.at[i, 'bag&backpack'])) + '.jpg'
    pathImg = os.path.join(pathBagBackpack, imgName)
    colorImg, _, _ = app_maskrcnn(pathImg)
    grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)

    hist = calcHistColorImg_(colorImg, (800, 1000))
    edgeFeature = calFeatureVector(grayscaleImg, (800, 1000))
    tmp.append([hist, edgeFeature])
  else:
    tmp.append(['NULL', 'NULL'])
###################################################################
  if not(isNan(df.at[i, 'hat'])):
    imgName = str(int(df.at[i, 'hat'])) + '.jpg'
    pathImg = os.path.join(pathHat, imgName)
    colorImg, _, _ = app_maskrcnn(pathImg)
    grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)

    hist = calcHistColorImg_(colorImg, (500, 500))
    edgeFeature = calFeatureVector(grayscaleImg, (500, 500))
    tmp.append([hist, edgeFeature])
  else:
    tmp.append(['NULL', 'NULL'])

  data_Recommend = tmp
  return data_Recommend

## **RÚT TRÍCH VÀ LƯU FILE**

In [28]:
for i in range(0, len(df)):
  data_Recommend = extractInfoImgs(df, i)

  path_recommend = '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/Data for Recommendation'
  name_file = str(i)
  name_file += '.txt'
  path_write = os.path.join(path_recommend, name_file)

  with open(path_write, 'wb') as f:
    np.save(f, np.asarray(data_Recommend))

  f.close()
  del data_Recommend
  del path_recommend
  del path_write
  del name_file

Clothes 0


KeyboardInterrupt: ignored

In [ ]:
with open(path_write, 'rb') as f:
  tmp = np.load(f, allow_pickle = True)

# **THỬ NGHIỆM**

## **ĐỌC ẢNH ĐẦU VÀO VÀ RÚT TRÍCH THÔNG TIN**

Ở phần này, ảnh đầu vào được xử lý tách nền sau đó mới rút trích thông tin

### **HÀM RÚT THÔNG TIN TỪ ẢNH ĐẦU VÀO**

In [27]:
def extractInfoImg(pathfile, nameImg):
  size=[(800, 1300), (800, 400), (800, 1000), (500, 500)]
  # top, bottom: 800, 1300
  # shoes: 800, 400
  # bag&backpack: 800, 1000
  # hat: 500, 500
  pathImg = os.path.join(pathfile, nameImg)
  Name_file = nameImg.split(".")[0]
  Name_file += '.txt'

  colorImg, sex, typeOutfit = app_maskrcnn(pathImg)
  grayscaleImg = cv.cvtColor(colorImg, cv2.COLOR_BGR2GRAY)
  
  if typeOutfit == 'top' or typeOutfit == 'bottom' or typeOutfit == 'dress':
    hist = calcHistColorImg_(colorImg, size[0])
    edgeFeature = calFeatureVector(grayscaleImg, size[0])
  elif typeOutfit == 'shoes':
    hist = calcHistColorImg_(colorImg, size[1])
    edgeFeature = calFeatureVector(grayscaleImg, size[1])
  elif typeOutfit == 'bag&backpack':
    hist = calcHistColorImg_(colorImg, size[2])
    edgeFeature = calFeatureVector(grayscaleImg, size[2])
  elif typeOutfit == 'hat':
    hist = calcHistColorImg_(colorImg, size[3])
    edgeFeature = calFeatureVector(grayscaleImg, size[3])

  Info = []
  Info.append(sex)
  Info.append([hist, edgeFeature])
  
  pathWrite = os.path.join(pathfile, Name_file)
  with open(pathWrite, 'wb') as f:
    np.save(f, np.asarray(Info))

  f.close()
  del hist
  del edgeFeature
  del colorImg

  return pathWrite, typeOutfit

In [33]:
pathfile = '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Data recommend'
listNameImg = os.listdir(pathfile)
pathInfo = []
typeOutfit = []
for NameImg in listNameImg:
  pathWrite, typeOutfit_ = extractInfoImg(pathfile, NameImg)
  pathInfo.append(pathWrite)
  typeOutfit.append(typeOutfit_)

Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_top.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_bottom.h5
Loading weights from  /gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Model/mask_rcnn_object_bottom.h5
Loading weights from  /gdrive/MyDri

In [34]:
for i in range(0, len(pathInfo)):
  tmp = pathInfo[i].split(os.path.sep)[-1].split("/")
  print(tmp[0] + ':' + typeOutfit[i])

1614.txt:top
1678.txt:top
1613.txt:top
1849.txt:top
1845.txt:top
7266.txt:bottom
6579.txt:bottom
4833.txt:bottom
4827.txt:bottom
6519.txt:bottom
6522.txt:bottom
5450.txt:bottom
4825.txt:bottom
4819.txt:bottom
1608.txt:bottom
1591.txt:bottom
1567.txt:bottom


## **XỬ LÝ VÀ CHỌN PHONG CÁCH PHỐI**

In [35]:
List = [['top'], ['bottom'], ['shoes'], ['bag&backpack'], ['hat']]

for i in range(0, len(pathInfo)):
  print('Outfit ', i)
  DataRecommend = os.listdir('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/Data for Recommendation')
  for j in DataRecommend:
    with open(pathInfo[i], 'rb') as f1:
      tmp1 = np.load(f1, allow_pickle = True)

    with open(os.path.join('/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Dataset/Data for Recommendation', j), 
              'rb') as f2:
      tmp2 = np.load(f2, allow_pickle = True)

    if tmp1[0] == tmp2[0]:
      if typeOutfit[i] == 'top' or typeOutfit[i] == 'dress':
        if tmp2[1][0] == 'NULL':
          continue
        cosin1 = cosinHistColor(tmp1[1][0], tmp2[1][0])
        cosin2 = cosinEdge(tmp1[1][1], tmp2[1][1])

      if typeOutfit[i] == 'bottom':
        if tmp2[2][0] == 'NULL':
          continue
        cosin1 = cosinHistColor(tmp1[1][0], tmp2[2][0])
        cosin2 = cosinEdge(tmp1[1][1], tmp2[2][1])

      if typeOutfit[i] == 'shoes':
        if tmp2[3][0] == 'NULL':
          continue
        cosin1 = cosinHistColor(tmp1[1][0], tmp2[3][0])
        cosin2 = cosinEdge(tmp1[1][1], tmp2[3][1])

      if typeOutfit[i] == 'bag&backpack':
        if tmp2[4][0] == 'NULL':
          continue
        cosin1 = cosinHistColor(tmp1[1][0], tmp2[4][0])
        cosin2 = cosinEdge(tmp1[1][1], tmp2[4][1])

      if typeOutfit[i] == 'hat':
        if tmp2[5][0] == 'NULL':
          continue
        cosin1 = cosinHistColor(tmp1[1][0], tmp2[5][0])
        cosin2 = cosinEdge(tmp1[1][1], tmp2[5][1])
    
    IDimg = pathInfo[i]
    IDimg = IDimg.split(os.path.sep)[-1].split("/")[0]
    IDimg = IDimg.split(".")[0]
    IDrecommend = j.split(".")[0]
    if cosin1 >= 0.7 and cosin2 >= 0.8:
      if typeOutfit[i] == 'top' or typeOutfit[i] == 'dress':
        List[0].append([IDrecommend, IDimg])
      if typeOutfit[i] == 'bottom':
        List[1].append([IDrecommend, IDimg])
      if typeOutfit[i] == 'shoes':
        List[2].append([IDrecommend, IDimg])
      if typeOutfit[i] == 'bag&backpack':
        List[3].append([IDrecommend, IDimg])
      if typeOutfit[i] == 'hat':
        List[4].append([IDrecommend, IDimg])
    # print('cosin Màu: ', cosin1)
    # print('cosin Cạnh: ', cosin2)
    # print()

Outfit  0
Outfit  1
Outfit  2
Outfit  3
Outfit  4
Outfit  5
Outfit  6
Outfit  7
Outfit  8
Outfit  9
Outfit  10
Outfit  11
Outfit  12
Outfit  13
Outfit  14
Outfit  15
Outfit  16


In [36]:
List

[['top',
  ['12', '1614'],
  ['16', '1614'],
  ['28', '1614'],
  ['0', '1613'],
  ['1', '1613'],
  ['4', '1613'],
  ['7', '1613'],
  ['10', '1613'],
  ['12', '1613'],
  ['13', '1613'],
  ['16', '1613'],
  ['20', '1613'],
  ['24', '1613'],
  ['25', '1613'],
  ['28', '1613'],
  ['31', '1613'],
  ['0', '1849'],
  ['4', '1849'],
  ['7', '1849'],
  ['10', '1849'],
  ['12', '1849'],
  ['13', '1849'],
  ['16', '1849'],
  ['19', '1849'],
  ['20', '1849'],
  ['24', '1849'],
  ['25', '1849'],
  ['28', '1849'],
  ['29', '1849'],
  ['31', '1849'],
  ['35', '1849'],
  ['1', '1845'],
  ['12', '1845'],
  ['16', '1845'],
  ['18', '1845'],
  ['20', '1845'],
  ['28', '1845'],
  ['31', '1845'],
  ['43', '1845'],
  ['44', '1845'],
  ['45', '1845'],
  ['46', '1845'],
  ['47', '1845'],
  ['48', '1845'],
  ['49', '1845'],
  ['50', '1845'],
  ['51', '1845'],
  ['52', '1845'],
  ['53', '1845'],
  ['54', '1845'],
  ['55', '1845'],
  ['56', '1845'],
  ['57', '1845'],
  ['58', '1845'],
  ['59', '1845'],
  ['60', 

In [37]:
pathFolderRecommendImg = '/gdrive/MyDrive/Đồ án/Đồ án Nhập môn Thị giác máy tính (CS231.M22.KHCL)/Data recommend'

for i in range(0, len(List)):
  for j in range(1, len(List[i])):
    Recommend = [List[i][j]]
    for k in range(i + 1, 5):
      for x in range(1, len(List[k])):
        if List[i][j][0] == List[k][x][0]:
          Recommend.append(List[k][x])
    if len(Recommend) < 2:
      continue

    pathList = []
    titleList = []
    for tmp in Recommend:
      pathList.append(os.path.join(pathFolderRecommendImg, tmp[1] + '.jpg'))
      titleList.append(tmp[1])
    showListImg(pathList, titleList)